## Import library

In [32]:
import numpy as np
import pandas as pd
import os
import warnings

import re
# import unidecode

from sklearn.feature_extraction.text import TfidfVectorizer

import lightgbm as lgb
from sklearn.metrics import roc_auc_score

warnings.filterwarnings(action="ignore")

DATA_PATH = "/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/"

In [5]:
df_train = pd.read_csv(DATA_PATH+"train.csv")
df_test = pd.read_csv(DATA_PATH+"test.csv")
df_sample = pd.read_csv(DATA_PATH+"sample_submission.csv")

## Preprocessing

In [6]:
# !pip install Unidecode

In [7]:
# 나는 구경도 못했는데 url이 있었다네 아니 뭔
'''
url_pattern = r"https?://\S+|www.\.\S+"
'''

'\nurl_pattern = r"https?://\\S+|www.\\.\\S+"\n'

In [8]:
# 그리고 unicode 변환은 또 뭐야
'''

'''

'\n\n'

In [9]:
# 첫번째 질문
# 그것보다는 불용어?처리를 하는게 좋지 않을까 싶었다. 추가적으로 하자면 .lower()처리정도?
# 그런데 BERT같은 transformer계열 모델을 사용한다면 attention으로 문장 내 단어간 관계을 알 수 있다.
# 그럼 제거 안해도 되나?

# 두번째 질문
# 간단한 방식, TF-IDF나 Word2Vec 같은걸 이용하는 방식도 해보고 싶다.
# 나는 이것들이 뭔지도 잘 모르니까
# 우리의 baseline은 그런 의미에서 간단하다.

In [19]:
def simple_clean_text(text):
    text = str(text)
    text.lower() # 같은 단어의 대,소문자 버전을 동일하게 인식하기 위해
    text = " ".join(text.split()) # 공백을 한칸으로만 깔끔하게 정리
    # TF-IDF나 Word2Vec 같은 임베딩은 공백(" ")을 기준으로 단어를 쪼갬(tokenize)
    # 공백이 2칸 이상이면 빈 문자열 생김
    return text

df_train['clean_text'] = df_train['comment_text'].apply(simple_clean_text)
df_test['clean_text'] = df_test['comment_text'].apply(simple_clean_text)

In [23]:
pd.set_option("display.max_colwidth", None)
df_train.loc[34:43][['comment_text']]

,comment_text
34,This bitch is nuts. Who would read a book by a woman.
35,Awesome!
36,Not for long! \n\n(Troll-In-Training since 2016)
37,Commenting for the sake of commenting to rate other comments to better comment on our new comment system.
38,Interesting concept.\n\nHow does Civil Comments plan to monetize their operation?
39,"Pity the menu lost so much vegan food! The Mash Tun was a favorite beer bar for all the delicious tempeh stuff.\nExcited the staff is still there, can't wait to try the new beers."
40,"I already see dozens of ""just commenting to vote on comments to be able to comment"" type of comments.\n\nThis voting system is completely insane- you're assuming people have the extra time to *actually read* multiple non related comments?\n\nI would love to see this system installed on oregonlive.com - they have enough online visitors and an established ""troll"" base to really need this service.\n\nWith as little online activity as wweek has - it will discourage further growth here."
41,Very cool project!
42,Awesome! Signed up just to give this a shot... good luck to your enterprise!
43,"disqus has way more functionality and obviously a huge following - it can increase visitor count significantly.\n\ndisqus also has a moderator feature - a feature that wweek either never wanted to pay their staff to undertake, or never developed a group of people they could trust to do it on their behalf on a volunteer basis."


In [24]:
df_train.loc[34:43][['clean_text']]

,clean_text
34,This bitch is nuts. Who would read a book by a woman.
35,Awesome!
36,Not for long! (Troll-In-Training since 2016)
37,Commenting for the sake of commenting to rate other comments to better comment on our new comment system.
38,Interesting concept. How does Civil Comments plan to monetize their operation?
39,"Pity the menu lost so much vegan food! The Mash Tun was a favorite beer bar for all the delicious tempeh stuff. Excited the staff is still there, can't wait to try the new beers."
40,"I already see dozens of ""just commenting to vote on comments to be able to comment"" type of comments. This voting system is completely insane- you're assuming people have the extra time to *actually read* multiple non related comments? I would love to see this system installed on oregonlive.com - they have enough online visitors and an established ""troll"" base to really need this service. With as little online activity as wweek has - it will discourage further growth here."
41,Very cool project!
42,Awesome! Signed up just to give this a shot... good luck to your enterprise!
43,"disqus has way more functionality and obviously a huge following - it can increase visitor count significantly. disqus also has a moderator feature - a feature that wweek either never wanted to pay their staff to undertake, or never developed a group of people they could trust to do it on their behalf on a volunteer basis."


In [25]:
# Embedding
tfidf = TfidfVectorizer(
    max_features = 50000, # 전체 단어 중 상위 5만개만 사용
    ngram_range = (1,2), # unigram(단일단어) + bigram(연속단어)
    max_df = 0.9, # 너무 자주 등장하는 단어 제외
    sublinear_tf = True # 단어 빈도에 로그 적용해 과도한 빈도 영향 완화
)

X_train = tfidf.fit_transform(df_train['clean_text'])
X_test = tfidf.fit_transform(df_test['clean_text'])

In [29]:
X_train[12293:12323].toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Baseline: LightGBM

In [ ]:
y = df_train['target'].values

params = {
    'objective':'regression',
    'metric':'auc',
    'boosting_type':'gbdt', # 학습방식 = {gbdt,dart,goss}
    'learning_rate':0.05,
    'n_estimators':500
}

lgbm = lgb.LGBMRegressor(**params)
lgbm.fit(X_train, y)

In [ ]:
y_pred = lgbm.predict(X_train)
print("Train ROC-AUC:", roc_auc_score())